In [47]:
import os
import numpy as np
from tqdm import *

In [48]:
data_path = [
    "data/POP909_train_12_measure.npy",
    "data/POP909_test_12_measure.npy",
    "data/POP909_validate_12_measure.npy"

    # "data/POP909_train_4_measure.npy",
    # "data/POP909_test_4_measure.npy",
    # "data/POP909_validate_4_measure.npy"
]

In [49]:
# import data
train_x = np.load(data_path[0],allow_pickle = True)
validate_x = np.load(data_path[1],allow_pickle = True)
test_x = np.load(data_path[2],allow_pickle = True)

In [50]:
# note extraction
hold_state = 128
rest_state = 129
def extract_note(x, pad_token = 128):
    d = []
    for i in x:
        if i < 128:
            d.append(i)
    ori_d = len(d)
    d.extend([pad_token] * (len(x) - len(d)))
    return np.array(d), ori_d

def extract_rhythm(x, hold_token = 2, rest_token = 3):
    d = []
    for i in x:
        if i < 128:
             d.append(1)
        elif i == hold_state:
             d.append(hold_token)
        else:
             d.append(rest_token)
    return np.array(d)

In [51]:
# process rhythm and pitch tokens
split_size = 24
new_data = []
# change here to be train_x/test_x/validate_x
for i,d in enumerate(train_x):
    d = np.array(d["notes"])
    ds = np.split(d, list(range(split_size,len(d),split_size)))
    data = []
    for sd in ds:
        if len(sd) != split_size:
            continue
        q,k = extract_note(sd)
        if k == 0:
            continue
        s = extract_rhythm(sd)
        data.append([sd,q,s,k])
    new_data.append(data)
    if i % 1000 == 0:
        print("processed:", i)

processed: 0


In [52]:
print(new_data[0])
print(len(new_data[0]))

[[array([ 69, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,  64, 128,
        64, 128,  64, 128,  69, 128, 128, 128,  71, 128,  71]), array([ 69,  64,  64,  64,  69,  71,  71, 128, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1,
       2, 1]), 7], [array([128, 128, 128,  69, 128,  68, 128,  69, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128,  64, 128,  64, 128,  64, 128]), array([ 69,  68,  69,  64,  64,  64, 128, 128, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2,
       1, 2]), 6], [array([128, 128,  69, 128,  71, 128, 128, 128,  71, 128, 128, 128,  69,
       128,  68, 128, 128, 128,  69, 128, 128, 128, 128, 128]), array([ 69,  71,  71,  69,  68,  69, 128, 128, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128

In [53]:
# we need to extract each measure in each song
final_data = []
for d in new_data:
    for dd in d:
        final_data.append(dd)
print(final_data[0])

[array([ 69, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,  64, 128,
        64, 128,  64, 128,  69, 128, 128, 128,  71, 128,  71]), array([ 69,  64,  64,  64,  69,  71,  71, 128, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1,
       2, 1]), 7]


In [54]:
res = []
# when training A4, num_measure should be 4.
# when training B4, num_measure should be 12.
num_measure = 4
length = int(len(final_data) / num_measure)
print(length)
for i in tqdm(range(length)):
    res.append(np.array(final_data[num_measure*i:num_measure*(i+1)]))
print((res[0]))
print(len(res[0]))


1614


  0%|          | 0/1614 [00:00<?, ?it/s]C:\Users\13273\AppData\Local\Temp\ipykernel_10260\1906833213.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res.append(np.array(final_data[num_measure*i:num_measure*(i+1)]))
100%|██████████| 1614/1614 [00:00<00:00, 146709.29it/s]

[[array([ 69, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,  64, 128,
          64, 128,  64, 128,  69, 128, 128, 128,  71, 128,  71])
  array([ 69,  64,  64,  64,  69,  71,  71, 128, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128])
  array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1,
         2, 1])
  7]
 [array([128, 128, 128,  69, 128,  68, 128,  69, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128,  64, 128,  64, 128,  64, 128])
  array([ 69,  68,  69,  64,  64,  64, 128, 128, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128])
  array([2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2,
         1, 2])
  6]
 [array([128, 128,  69, 128,  71, 128, 128, 128,  71, 128, 128, 128,  69,
         128,  68, 128, 128, 128,  69, 128, 128, 128, 128, 128])
  array([ 69,  71,  71,  69,  68,  69, 128, 128, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 

In [55]:
# change the name
np.save("data/POP909_train_12_chord_rhythm.npy", res)
# np.save("data/POP909_test_12_chord_rhythm.npy", res)
# np.save("data/POP909_validate_12_chord_rhythm.npy", res)

# np.save("data/POP909_train_4_chord_rhythm.npy", res)
# np.save("data/POP909_test_4_chord_rhythm.npy", res)
print(res[0])
# np.save("data/POP909_validate_4_chord_rhythm.npy", res)

[[array([ 69, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,  64, 128,
          64, 128,  64, 128,  69, 128, 128, 128,  71, 128,  71])
  array([ 69,  64,  64,  64,  69,  71,  71, 128, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128])
  array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1,
         2, 1])
  7]
 [array([128, 128, 128,  69, 128,  68, 128,  69, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128,  64, 128,  64, 128,  64, 128])
  array([ 69,  68,  69,  64,  64,  64, 128, 128, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128])
  array([2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2,
         1, 2])
  6]
 [array([128, 128,  69, 128,  71, 128, 128, 128,  71, 128, 128, 128,  69,
         128,  68, 128, 128, 128,  69, 128, 128, 128, 128, 128])
  array([ 69,  71,  71,  69,  68,  69, 128, 128, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 